# Ternarised Convolutional Neural Networks

This tutorial demonstrates training a simple Ternarised Convolutional Neural Network to classify MNIST digits. 

### Import TensorFlow and Larq

In [1]:
import tensorflow as tf
import larq as lq

### Download and prepare the MNIST dataset

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between -1 and 1
train_images, test_images = train_images / 127.5 - 1., test_images / 127.5 - 1.

### Create the model

In [3]:
kwargs = dict(input_quantizer=lq.quantizers.SteTern(), kernel_quantizer=lq.quantizers.SteTern(ternary_weight_networks=True), kernel_constraint="weight_clip")

model = tf.keras.models.Sequential()

model.add(lq.layers.QuantConv2D(32, (3, 3), kernel_quantizer=lq.quantizers.SteTern(), kernel_constraint="weight_clip", use_bias=False, input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Flatten())

model.add(lq.layers.QuantDense(64, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(lq.layers.QuantDense(10, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Activation("softmax"))

 Here's the complete architecture of our model.
 
 Almost all parameter of the network a binarized (either -1 or 1). This will make the network extremly fast when deployed on a embedded device that supports binarized neural networks.

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quant_conv2d (QuantConv2D)   (None, 26, 26, 32)        288       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 13, 13, 32)        96        
_________________________________________________________________
quant_conv2d_1 (QuantConv2D) (None, 11, 11, 64)        18432     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 5, 64)          192       
_________________________________________________________________
quant_conv2d_2 (QuantConv2D) (None, 3, 3, 64)          36864     
__________

### Compile and train the model

Note: This may take a few minutes depending on your system.

In [5]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, batch_size=50, epochs=10)

test_loss, test_acc = model.evaluate(test_images, test_labels)

Epoch 1/10
60000/60000 [==============================] - 8s 130us/step - loss: 0.6090 - acc: 0.9234
Epoch 2/10
60000/60000 [==============================] - 6s 101us/step - loss: 0.4646 - acc: 0.9664
Epoch 3/10
60000/60000 [==============================] - 6s 101us/step - loss: 0.4407 - acc: 0.9731
Epoch 4/10
60000/60000 [==============================] - 6s 101us/step - loss: 0.4343 - acc: 0.9755
Epoch 5/10
60000/60000 [==============================] - 6s 101us/step - loss: 0.4256 - acc: 0.9779
Epoch 6/10
60000/60000 [==============================] - 6s 100us/step - loss: 0.4227 - acc: 0.9794
Epoch 7/10
60000/60000 [==============================] - 6s 101us/step - loss: 0.4218 - acc: 0.9800
Epoch 8/10
60000/60000 [==============================] - 6s 101us/step - loss: 0.4171 - acc: 0.9814
Epoch 9/10
60000/60000 [==============================] - 6s 101us/step - loss: 0.4141 - acc: 0.9830
Epoch 10/10
10000/10000 [==============================] - 1s 56us/step


### Evaluate the model

In [6]:
print(f"Test accuracy {test_acc * 100:.2f} %")

Test accuracy 98.52 %


As you can see, our simple binarized CNN has achieved a test accuracy of over 97.5 %. Not bad for a few lines of code!